<a href="https://colab.research.google.com/github/itsvishal07/Mobile_Computing/blob/main/GSM_and_GSM_architechture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# STATIC PARAMETERS
NUM_TRIALS = 10000
PRICE_PER_CALL = 0.5

# Zone technical parameters and cost/revenue data
ZONE_PARAMS = [
    {"name": "Zone 1", "RADIUS": 17, "SIDE": 25, "CHANNELS_PER_CELL": 4, "REUSE_FACTOR": 100},
    {"name": "Zone 2", "RADIUS": 33, "SIDE": 5, "CHANNELS_PER_CELL": 81},
    {"name": "Zone 3", "RADIUS": 15, "SIDE": 5, "CHANNELS_PER_CELL": 31},
]

ZONE_DATA = [
    {"name": "Zone 1", "subs": 10000},
    {"name": "Zone 2", "subs": 15000},
    {"name": "Zone 3", "subs": 5000},
]

REVENUE_ASSUMPTIONS = {
    "AVG_CALL_DURATION": 2.4,
    "VOICE_RATE_PER_MIN": 0.5,
    "AVG_SMS": 3,
    "SMS_RATE": 0.2,
    "AVG_DATA_GB": 1.5,
    "DATA_RATE_PER_GB": 10,
    "VAS_RATE": 5,
    "VAS_RATIO": 0.2,
    "PREPAID_RATIO": 0.6,
    "INTERNET_PACK_RATE": 3,  # corrected key from 'INTERNET PACK RATE'
    "ROAM_RATE": 15,
    "LEASE_RATE": 200,
    "ADS_RATE": 0.1,
    "ENTERPRISE_CLIENTS": [15, 20, 3],
    "ENTERPRISE_RATE": 500
}

def generate_walsh_matrix(size):
    if size == 1:
        return np.array([[1]])
    H_prev = generate_walsh_matrix(size // 2)
    return np.vstack((np.hstack((H_prev, H_prev)), np.hstack((H_prev, -H_prev))))

# Simulation Parameters
num_users = 8
chip_rate = 1_000_000  # 1 Mbps
spreading_factor = 8
data_rate = chip_rate / spreading_factor
num_bits = 100

# Generate Walsh codes
walsh_matrix = generate_walsh_matrix(spreading_factor)
user_codes = walsh_matrix[:num_users]

# User data
user_data = np.random.randint(0, 2, size=(num_users, num_bits)) * 2 - 1
transmitted_signals = []
for i in range(num_users):
    spread_signal = np.repeat(user_data[i], spreading_factor) * np.tile(user_codes[i], num_bits)
    transmitted_signals.append(spread_signal)

# AWGN channel
noise_power_db = -10
noise_power_linear = 10 ** (noise_power_db / 10)
noise = np.sqrt(noise_power_linear) * np.random.randn(len(transmitted_signals[0]))
received_signal = np.sum(transmitted_signals, axis=0) + noise

# Receiver
decoded_data = np.zeros_like(user_data)
snr_db = np.zeros(num_users)
for i in range(num_users):
    despread = received_signal.reshape(num_bits, spreading_factor) @ user_codes[i]
    decoded_data[i] = np.sign(despread)
    sig_power = np.mean(transmitted_signals[i] ** 2)
    intf = sum([transmitted_signals[j] for j in range(num_users) if j != i])
    intf_power = np.mean(intf ** 2)
    snr_db[i] = 10 * np.log10(sig_power / (intf_power + noise_power_linear))

# Display Results
print("\n--- Simulation Summary ---")
for i in range(num_users):
    match = "Match" if np.array_equal(user_data[i], decoded_data[i]) else "Mismatch"
    print(f"User {i+1}: {match}, SNR: {snr_db[i]:.2f} dB")

print("\nAverage SNR:", np.mean(snr_db), "dB")



--- Simulation Summary ---
User 1: Match, SNR: -8.51 dB
User 2: Match, SNR: -8.51 dB
User 3: Match, SNR: -8.51 dB
User 4: Match, SNR: -8.51 dB
User 5: Match, SNR: -8.51 dB
User 6: Match, SNR: -8.51 dB
User 7: Match, SNR: -8.51 dB
User 8: Match, SNR: -8.51 dB

Average SNR: -8.512583487190753 dB
